# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [29]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_nhl():
    nhl_df=pd.read_csv("assets/nhl.csv")
    
    # Filter 2018 year
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    
    # remove uneccessary columns
    nhl_df = nhl_df[['team', 'W', 'L']]

    # clean up team names
    def remove_star(string):
        if '*' in string:
            return re.findall("([\w\s]*)", string)[0]
        else:
            return string
    nhl_df['team'] = nhl_df.team.apply(remove_star)

    # extract team names in a column called 'team_name'
    nhl_df['team'] = nhl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nhl_df.loc[3,'team'] = "Maple Leafs"
    nhl_df.loc[5,'team'] = "Red Wings"
    nhl_df.loc[13,'team'] = "Blue Jackets"
    nhl_df.loc[27,'team'] = "Golden Knights"
    
    nhl_df.loc[14, 'team'] = "Rangers Islanders Devils"
    nhl_df.loc[16, 'team'] = "Rangers Islanders Devils"
    nhl_df.loc[17, 'team'] = "Rangers Islanders Devils"
    
    nhl_df.loc[28, 'team'] = "Kings Ducks"
    nhl_df.loc[30, 'team'] = "Kings Ducks"
    
    # dropping entries with the word 'Divison' from nhl_df
    nhl_df = nhl_df[nhl_df["team"].str.contains("Division") == False]

    # change W L data type to float for computation
    nhl_df["W"] = nhl_df["W"].astype("float")
    nhl_df["L"] = nhl_df["L"].astype("float")
    
    #  find avg w/l for the teams with same name
    nhl_df = nhl_df.groupby('team')['W', 'L'].mean()
    # generate w/l ratio
    nhl_df['W-L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    nhl_df.reset_index(inplace = True)

    return nhl_df

def clean_wiki():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean up team names in wiki site
    def remove_braces(string):
        if '[' in string:
            return re.findall("([\w\s]*)", string)[0]
        else:
            return string
    cities['NHL'] = cities.NHL.apply(remove_braces)
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
    
    # rename cities df columns names to match nhl_df & sort city name alphabetically
    cities.rename(columns={'Metropolitan area':'city', 'Population (2016 est.)[8]':'population', 'NHL':'team_name'}, inplace = True)
    
    return cities

def nhl_correlation(): 
    # YOUR CODE HERE
    # combine 2 data frames
    merge = pd.merge(clean_nhl(), clean_wiki(), how='inner', on='team_name')
        
    population_by_region = pd.to_numeric(merge['population'], errors = 'coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merge['W-L%'], errors = 'coerce') # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
#nhl_correlation()
clean_nhl()

,team,W,L,W-L%
0,Avalanche,43.000000,30.0,0.589041
1,Blackhawks,33.000000,39.0,0.458333
2,Blue Jackets,45.000000,30.0,0.600000
3,Blues,44.000000,32.0,0.578947
4,Bruins,50.000000,20.0,0.714286
5,Canadiens,29.000000,40.0,0.420290
6,Canucks,31.000000,40.0,0.436620
7,Capitals,49.000000,26.0,0.653333
8,Coyotes,29.000000,41.0,0.414286
9,Flames,37.000000,35.0,0.513889


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_nba():
    nba_df=pd.read_csv("assets/nba.csv")
    # YOUR CODE HERE
    nba_df = nba_df[nba_df['year'] == 2018]
    
    # clean team name in nba_df
    nba_df['team'] = nba_df['team'].apply(lambda x: re.findall('([\w\s]*)', x)[0] if '*' or '(' in x else x)
    nba_df['team'] = nba_df['team'].apply(lambda y: y.rsplit(None, 1)[-1])
    nba_df.loc[17, 'team'] = 'Trail Blazers'
    
    nba_df.loc[10, 'team'] = 'Knicks Nets'
    nba_df.loc[11, 'team'] = 'Knicks Nets'
    nba_df.loc[24, 'team'] = 'Lakers Clippers'
    nba_df.loc[25, 'team'] = 'Lakers Clippers'
    
    # discard uneccesary columns from nba_df
    nba_df = nba_df[['team', 'W/L%']] 
    
    # change datatypes
    nba_df['W-L%'] = nba_df['W/L%'].astype(float)
    
    # group by team
    nba_df = nba_df.groupby('team')['W-L%'].mean()
    nba_df = nba_df.reset_index()
    return nba_df


def clean_wiki():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean team name in cities
    cities['NBA'] = cities['NBA'].apply(lambda x: re.findall('([\w\s]*)', x)[0] if '[' in x else x)
    
    # discard uneccessary columns from cities
    cities = cities[['Population (2016 est.)[8]', 'NBA']]
    
    # rename columns for joining 
    cities.rename(columns = {'Population (2016 est.)[8]':'Population', 'NBA':'team'}, inplace = True)
    
    return cities

    
def nba_correlation():
    # merge 2 df
    merged = clean_wiki().merge(clean_nba(), how='inner', on='team')     
    population_by_region = pd.to_numeric(merged['Population'], errors='coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged['W-L%'], errors='coerce') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

#nba_correlation()
clean_nba()

,team,W-L%
0,76ers,0.6340
1,Bucks,0.5370
2,Bulls,0.3290
3,Cavaliers,0.6100
4,Celtics,0.6710
5,Grizzlies,0.2680
6,Hawks,0.2930
7,Heat,0.5370
8,Hornets,0.4390
9,Jazz,0.5850


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [32]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_mlb():
    mlb_df=pd.read_csv("assets/mlb.csv")
    
    # year 2018
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    
    # discard uneccesary columns from mlb_df
    mlb_df = mlb_df[['team', 'W-L%']] 
    
    # find team name
    mlb_df['team'] = mlb_df['team'].apply(lambda y: y.rsplit(None, 1)[-1])
    
    mlb_df.loc[13, 'team'] = 'Dodgers Angels'
    mlb_df.loc[25, 'team'] = 'Dodgers Angels'
    
    mlb_df.loc[18, 'team'] = 'Yankees Mets'
    mlb_df.loc[1, 'team'] = 'Yankees Mets'
    
    mlb_df.loc[11, 'team'] = 'Giants Athletics'
    mlb_df.loc[28, 'team'] = 'Giants Athletics'
    
    mlb_df.loc[0, 'team'] = 'Red Sox'
    mlb_df.loc[3, 'team'] = 'Blue Jays'
    
    mlb_df.loc[21, 'team'] = 'Cubs White Sox'
    mlb_df.loc[8, 'team'] = 'Cubs White Sox'
    
    # groupby team name
    mlb_df = mlb_df.groupby('team')['W-L%'].mean()
    mlb_df = mlb_df.reset_index()
    
    return mlb_df

def clean_wiki():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean team name in cities
    cities['MLB'] = cities['MLB'].apply(lambda x: re.findall('([\w\s]*)', x)[0] if '[' in x else x)
    
    # discard uneccessary columns from cities
    cities = cities[['Population (2016 est.)[8]', 'MLB']]
    
    # rename columns for joining 
    cities.rename(columns = {'Population (2016 est.)[8]':'Population', 'MLB':'team'}, inplace = True)
    return cities


def mlb_correlation(): 
    # YOUR CODE HERE
    # merge 2 dataframe
    merged = clean_wiki().merge(clean_mlb(), how='inner', on='team')
    
    population_by_region = pd.to_numeric(merged['Population'], errors='coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged['W-L%'], errors='coerce') # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

#mlb_correlation()
clean_mlb()

,team,W-L%
0,Astros,0.636
1,Blue Jays,0.451
2,Braves,0.556
3,Brewers,0.589
4,Cardinals,0.543
5,Cubs White Sox,0.483
6,Diamondbacks,0.506
7,Dodgers Angels,0.529
8,Giants Athletics,0.525
9,Indians,0.562


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_nfl():
    nfl_df=pd.read_csv("assets/nfl.csv")
    # year 2018
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    
    # discard uneccessary columns
    nfl_df = nfl_df[['W', 'L', 'team']]
    
    # dropping entries with the pattern 'FC' from nhl_df
    nfl_df = nfl_df[nfl_df["W"].str.contains("FC") == False]
    
    # clean team name
    nfl_df['team'] = nfl_df['team'].apply(lambda x: re.findall('([\w\s]*)', x)[0] if '*' or '+' in x else x)
    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.rsplit(None, 1)[-1])
    
    nfl_df.loc[4, 'team'] = 'Giants Jets'
    nfl_df.loc[24, 'team'] = 'Giants Jets'
    
    nfl_df.loc[17, 'team'] = 'Rams Chargers'
    nfl_df.loc[36, 'team'] = 'Rams Chargers'
    
    nfl_df.loc[38, 'team'] = '49ers Raiders'
    nfl_df.loc[19, 'team'] = '49ers Raiders'
    
    # group by team name
    nfl_df['W'] = nfl_df['W'].astype(float)
    nfl_df['L'] = nfl_df['L'].astype(float)
    nfl_df = nfl_df.groupby('team')['W', 'L'].mean()
    
    # create win-loss ratio
    nfl_df['W-L%'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    nfl_df.reset_index(inplace=True)
    
    return nfl_df

def clean_wiki():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # discard uneccessary columns 
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    cities.rename(columns = {'Population (2016 est.)[8]':'Population', 'NFL':'team'}, inplace = True)
    
    # clean team names
    cities['team'] = cities['team'].apply(lambda x: re.findall('([\w\s]*)', x)[0] if '[' in x else x)
    
    return cities


def nfl_correlation(): 
    # YOUR CODE HERE
    merged = clean_wiki().merge(clean_nfl(), on='team', how='inner')
    
    population_by_region = pd.to_numeric(merged['Population'], errors='coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged['W-L%'], errors='coerce') # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

#clean_nfl()
#clean_wiki()
nfl_correlation()

0.004922112149349414

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [49]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_dfs():
    nfl_df = clean_nfl()
    nba_df = clean_nba()
    nhl_df = clean_nhl()   
    mlb_df = clean_mlb()
    
    
    nfl_df = nfl_df[["team", "W-L%"]]
    nba_df = nba_df[["team", "W-L%"]]
    nhl_df = nhl_df[["team", "W-L%"]]  
    mlb_df = mlb_df[["team", "W-L%"]]
    
    return (nfl_df, nba_df, nhl_df, mlb_df)

def cal_p_values(sports):
    pvalues = pd.DataFrame(columns=sports.keys(), index=sports.keys())
    for sport1, sport1_df in sports.items():
        for sport2, sport2_df in sports.items():
            merged_league = pd.merge(sport1_df, sport2_df, on="team")
            pvalues.loc[sport1, sport2] = stats.ttest_rel(merged_league["W-L%_x"], merged_league["W-L%_y"])[1]
    return pvalues

(nfl_df, nba_df, nhl_df, mlb_df) = clean_dfs()
sports = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}



def sports_team_performance():
    # YOUR CODE HERE
    (nfl_df, nba_df, nhl_df, mlb_df) = clean_dfs()
    sports = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}

    p_values_dic = cal_p_values(sports).astype(float)
    p_values = pd.DataFrame(p_values_dic).astype(float)
        
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    return nfl_df
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    

sports_team_performance()
#p_values(sports)
#clean_dfs()
#clean_nfl()

,team,W-L%
0,49ers Raiders,0.250000
1,Bears,0.750000
2,Bengals,0.375000
3,Bills,0.375000
4,Broncos,0.375000
5,Browns,0.466667
6,Buccaneers,0.312500
7,Cardinals,0.187500
8,Chiefs,0.750000
9,Colts,0.625000
